数据源：mongodb170

## 用户信息 

In [ ]:
import pymongo
import pandas as pd
From LAC import LAC
import time

In [ ]:
db = pymongo.MongoClient('mongodb://192.168.11.170:27017')['weibo']                              #连接mongodb170数据
user_info = db['user'].find()                                                                    #获取用户信息的表格
user_info = pd.DataFrame(user_info)                                                              #将获取到的表格转化为dataframe格式
user_info
# user = db['user'].find({}, {'_id': 1, 'fan_num': 1, 'follow_num': 1,'tweet_num': 1})
# user = pd.DataFrame(user)

In [ ]:
user_info_o = user_info.drop(['crawl_at','crawl_at_timestamp','head_img_url','v'],
                             axis=1,inplace=False)                                               #删除分析不需要的列
user_info_o.rename(columns={'_id':'user_id'},inplace = True)                                     #重命名
user_info_o

In [ ]:
user_info_o.columns                                                                               #显示各列名（用户的所有属性）

In [ ]:
user_info_o.isna().sum()                                                                          #各列缺失值统计

除用户ID以及用户类型之外，其他十六组变量都存在缺失值；

In [ ]:
tweet_info = db['tweet_user'].find()
tweet_info = pd.DataFrame(tweet_info)
tweet_info                                                                                         #加载用户帖子的信息
# user 

In [ ]:
len(tweet_info['user_id'].unique())                                                                #68W帖子对应的用户数目

In [ ]:
u_c = tweet_info.groupby('user_id').apply(lambda x: ''.join(x['content']+' '))                    #将每个用户发的所有帖子聚合到一起
u_c = pd.DataFrame(u_c)
u_c.columns = ['content']
u_c

In [ ]:
pd_con = pd.merge(u_c,user_info_o,on='user_id',how = 'inner')                                      #将帖子和用户信息两个表格merge，通过相同列user_id
pd_con

In [ ]:
pd_con.isna().sum()                                                                                 #合并后表格的缺失值统计

对于这2166条数据，个人简介、生日、标签、城市、受教育程度、认证信息、性取向、职业、情感状态存在缺失值；

有2条数据粉丝数目，关注信息以及发微博数缺失，认定为异常值。

### 文本清洗（Xu）

In [ ]:
def clean_text(df,param1 = False,param2 = True):
    """
    固定删除文本中的日期，时间，邮件地址，网址，“转发微博”，“显示地图”，“分享图片”
    “XXX的微博视频”，含“谦毅”的帖子，选择删除表情，标签
    :param df: dataframe, the content we need to clean is df['content']
    :param param1: 是否删除文中的【】中的表情
    :param param2: 是否删除文中#键间的文字
    :return: 清洗过后的dataframe文本
    """
    df = df.astype(str)                                                                           # 将所有元素转化为str类型
    df['content'] = df['content'].str.replace(r"\d+/\d+/\d+|", '') \
                                 .str.replace("【",'').str.replace("】",'')\
                                 .str.replace(r"[0-2]?[0-9]:[0-6][0-9]", '') \
                                 .str.replace(r"[\w]+@[\.\w]+", '') \
                                 .str.replace(u"@(.*?):|/@(.*) /", '') \
                                 .str.replace(r"([hH][tT]{2}[pP]://|[hH][tT]{2}[pP][sS]://|[wW]{3}."
                                              r"|[wW][aA][pP].|[fF][tT][pP].|[fF][iI][lL][eE].)[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]",'')\
                                 .str.replace(u"谦毅:(.*?)|/转发微博/g|/显示地图/g|/分享图片/g",'')\
                                 .str.replace(u"#\S*?的微博视频|:\S*?的微博视频",'')\
                                 .str.replace(u'\[.*?]|#| \S*? 显示地图|转发微博|分享图片| \S*?的微博视频|//@.*?:|@\S*? |@\S*?$','')

    if param1:
        df['content'] = df['content'].str.replace(u"\\[.*?]",'')                                    # 删除【】和【】中的表情
    if param2:
        df['content'] = df['content'].str.replace(u"\\#.*?#",'')                                    # 删除#号和#间的话题

    return df

In [ ]:
df2 = clean_text(pd_con,param1 = True,param2 = True)
df2

### 对聚合后的帖子进行分词以及词性标注 

In [ ]:
def lac_text_baidu(sentence,tag_list):
    '''
    词性标注并筛选（百度lac）
    '''
    stopwords = stopwordslist()
#    lac.load_customization('mydict.txt')
    lac_t = lac_lac.run(sentence)
    data = {'word':lac_t[0],'tag':lac_t[1]}
    df = pd.DataFrame(data)
    df1 = df['word'].loc[df['tag'].isin(tag_list)]
    outstr = []
    for i in df1:
        if i not in stopwords and len(i ) >1:
            outstr.append(i)
    return outstr

def lac_seq_baidu(sentence):
    stopwords = stopwordslist()
    lac_t = lac_lac.run(sentence)
    data = {'word':lac_t[0],'tag':lac_t[1]}
    df = pd.DataFrame(data)
    df1 = df['word']
    outstr = []
    for i in df1:
        if i not in stopwords and len(i)>1:
            outstr.append(i)
    return outstr

def stopwordslist():
    '''
    导入停词表文件
    '''
    stopwords = [line.strip() for line in open('停用词表全.txt',encoding='UTF-8').readlines()]
    return stopwords

In [ ]:
df2['content'][8]

In [ ]:
from LAC import LAC
lac_lac = LAC(mode='lac')

- lac分词各个标签的含义

|标签	|含义	|标签	|含义	|标签	|含义	|标签	|含义|
|:---|:---|:---|:---|:---|:---|:---|:---|
|n	|普通名词	|f	|方位名词	|s	|处所名词	|nw	|作品名|
|nz	|其他专名	|v	|普通动词	|vd	|动副词	|vn	|名动词|
|a	|形容词	|ad	|副形词	|an	|名形词	|d	|副词|
|m	|数量词	|q	|量词	|r	|代词	|p	|介词|
|c	|连词	|u	|助词	|xc	|其他虚词	|w	|标点符号|
|PER	|人名	|LOC	|地名	|ORG	|机构名	|TIME	|时间|

In [ ]:
import time
start_time = time.time()


df2['content_seq'] = df2['content'].apply(lambda x:lac_seq_baidu(x))                                 #lac分词结果  
df2['content_select']  = df2['content'].apply(lambda x:lac_text_baidu(x,['n','nz','vn',
                                                                         's','LOC','PER','ORG']))     #lac分词结果筛选，去除停用词
'''
筛选出名词、其他专名、名动词、处所名词、地名、人名、机构名
'''
end_time = time.time()
print(end_time-start_time)

In [ ]:
df2                                                                                     #添加分词结果后的表格

- 

In [ ]:
df2.to_csv('发帖用户.csv')                                                             #将df2输出到csv，再转化为Excel文件，对数据有个直观的概览

In [ ]:
df3 = df2[['user_id','content_seq','content_select']]
df3                                                                                    #表格df3为user_id及对应的文本

In [ ]:
df3['content_select'][2]

In [ ]:
a = []
for i in range(len(df3)):
    if len(df3['content_select'][i]) == 0:
        a.append(i)                                                                    #识别帖子内容为空的用户对应的index
df4 = df3.drop(labels=None,axis=0,index=a)                                            #删除帖子内容为空的用户 默认inplace=false
df4.reset_index(drop=True)                                                            #重新排列索引

In [ ]:
df4.to_csv('用户帖子分词.csv')